In [1]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.types import CachePolicy
from langgraph.cache.memory import InMemoryCache
from datetime import datetime

In [2]:
def reducer_func(old: list[str], new: list[str]):
    return old + new

In [3]:
import operator


class State(TypedDict):
    time: str
    # messages: Annotated[list[str], reducer_func]
    # messages: Annotated[list[str], operator.add]


graph_builder = StateGraph(
    State,
)

In [4]:
def node_one(state: State) -> State:
    messages = state.get("messages", [])

    response = "Hello, nice to meet you"

    return {"messages": [response]}


def node_two(state: State):

    return {"time": f"{datetime.now()}"}


def node_three(state: State) -> State:

    return {}

In [5]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node(
    "node_two",
    node_two,
    cache_policy=CachePolicy(
        ttl=20,  # Node will be cached for 20 secs / Use when node is expensive
    ),
)
graph_builder.add_node("node_three", node_three)


graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", END)

In [6]:
import time

graph = graph_builder.compile(cache=InMemoryCache())


print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))

{'time': '2025-11-20 13:32:19.227530'}
{'time': '2025-11-20 13:32:19.227530'}
{'time': '2025-11-20 13:32:19.227530'}
{'time': '2025-11-20 13:32:19.227530'}
{'time': '2025-11-20 13:32:39.250496'}
